<a href="https://colab.research.google.com/github/SabtheCrab/CS-175-Final-Project/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CS 175 Final Project: Humback Whale Identification
Group Member 1:

Name: Sabrina Will

Group Member 2:

Name: Jeffrey Lee Ye

CREDITS FOR BOUNDING BOX CROPPED DATASET: https://www.kaggle.com/josemontiel/humpback-whale-bounding-box-cropped-dataset/output

#Loading the Data in

In [4]:
#We are using a data set, which originally came from the kaggle competition. However, many of the images suffered from misalignment issues and have 
#large amounts of background space we dont need our AI to consider when looking at whale flukes. As such, someone blessed competitors with a
#dataset that reformats all original images and crops out just the whale fluke itself. This is the data we are using in our project. 

from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
%cd /content/drive/My\ Drive/project/data

/content/drive/My Drive/project/data


In [73]:
!ls

 checkpoint.pth.tar   train.csv  'training (1)'


In [74]:
import copy, cv2, json, os, time, torch, torchvision

from collections import OrderedDict

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import pandas as pd
from PIL import Image
from torch import nn
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.utils.data as data
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
import torchvision.models as models
from torchvision import transforms
import torchvision.datasets as datasets

import pandas as pd

import seaborn as sns

df = pd.read_csv('train.csv')
df.head()

,Image,Id
0,fee74461e.jpg,w_314bc30
1,feea8c488.jpg,w_5b16177
2,feed76893.jpg,w_bdded04
3,fef0dc4a1.jpg,w_e6325d7
4,fef49fdb3.jpg,new_whale


This is what the training csv looks like. It contains an image and the correct whale id associated with the image

In [75]:
#find number of unique whale individuals, we are only importing the first 100 unique individuals for the demo
n_unique = len(df['Id'].unique())
print ("Number of Unique Categories : " + str(n_unique))

Number of Unique Categories : 58


In [0]:
class HumpbackWhale_Dataset(Dataset):
    def __init__(self,filepath, csv_path, transform=None):
        self.file_path = filepath
        self.df = pd.read_csv(csv_path)
        self.transform = transform
        self.image_list = [x for x in os.listdir(self.file_path)]
        
    def __len__(self):
        return(len(self.image_list))
    
    def __getitem__(self,idx):
        img_path = os.path.join(self.file_path,self.df.Image[idx])
        label = self.df.Id[idx]
        imgRGB = Image.open(img_path).convert('RGB')
        img = self.transform(imgRGB)
        imgName = self.df.Image[idx]
        return img, label, imgName

def label_to_id(label):
    x = [label_to_idNum[i] for i in label]
    return torch.tensor(x)

In [0]:
dims = 128

transform = transforms.Compose([
                              transforms.Resize((dims, dims)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

In [79]:
train_dataset = HumpbackWhale_Dataset('training (1)','train.csv', transform)
print(len(train_dataset.image_list))

100


In [80]:
test_size = .2
n = len(train_dataset)

np.random.seed(0)
a = list(range(n))

train_index = np.random.choice(a,replace=False,size=int(n*(1-test_size)))
test_index = np.setdiff1d(a,train_index)
print(train_index.size)
print(test_index.size)

80
20


In [81]:
#load the data in finally
data_train = copy.deepcopy(train_dataset)
data_train.image_list = [train_dataset.image_list[i] for i in train_index]

data_validate = copy.deepcopy(train_dataset)
data_validate.image_list = [train_dataset.image_list[i] for i in test_index]

gen_train = DataLoader(data_train,batch_size=20, shuffle=True)
gen_validate = DataLoader(data_validate,batch_size=20, shuffle=True)

print(len(gen_train))
print(len(gen_validate))

4
1


In [82]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=5005):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(int(32*dims*dims/4/4), num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

print("Cuda" if torch.cuda.is_available() else "CPU")
  
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = ConvNet(n_unique).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Cuda


In [83]:
n_train = len(df['Id'])
#counts of each unique whale
weight_series = df['Id'].value_counts()
weight_df = pd.DataFrame(weight_series)
weight_df.columns = ['count']
weight_df['count'] = weight_df['count'].astype(float)
weight_df['Id'] = weight_df.index
weight_df = weight_df.reset_index(drop = True)
weight_df['weight'] = n_train / (n_unique * weight_df['count'] )
label_to_idNum = dict(zip(weight_df['Id'], range(n_unique)))
weight_df

,count,Id,weight
0,42.0,new_whale,0.041051
1,2.0,w_f765256,0.862069
2,1.0,w_bfae26b,1.724138
3,1.0,w_bdded04,1.724138
4,1.0,w_772845b,1.724138
5,1.0,w_fbcfe74,1.724138
6,1.0,w_3f93ee8,1.724138
7,1.0,w_9f30885,1.724138
8,1.0,w_ba4f96c,1.724138
9,1.0,w_b3e445e,1.724138


In [92]:
# Train data on part of the full training data to check performance with
epochs = 8
# training_data = full_train_generator
training_data = gen_train

image_count = len(training_data)
progress_printer = 25

for e in range(epochs):
    running_loss, i = 0, 0

    for image, label, _ in training_data:
        label = label_to_id(label)
        image, label = image.to(device), label.to(device)
        i +=1 
        if i % progress_printer == 0:
            print('{:.0f}% complete'.format(i/image_count*100))
        log_ps = model(image)
        loss = criterion(log_ps, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print('epoch {}: loss: {}'.format(e,running_loss))

epoch 0: loss: 1.9859444200992584
epoch 1: loss: 0.7412857860326767
epoch 2: loss: 0.20648090913891792
epoch 3: loss: 0.19542932603508234
epoch 4: loss: 0.028489333228208125
epoch 5: loss: 0.2931515444070101
epoch 6: loss: 0.46714760549366474
epoch 7: loss: 0.22495484357932582


In [0]:
def testModel(model, testing_data):
    model.eval()
    with torch.no_grad():
      numCorrect = 0
      total = 0
      batch = 0 
      progress_printer = 25
      for image, label,imageName in testing_data:
          label = label_to_id(label).to(device)
          image = image.to(device)
          predicted = model(image)
          top5Pred, top5PredIndices = torch.topk(predicted, 5)
          #top5Pred, top5PredIndices = torch.topk(predicted, 1)
          for p in range(len(top5PredIndices)):
              currPred = top5PredIndices[p]
              numCorrect += (label[p] in currPred)
          total += len(label)
          batch += 1
          if (batch % progress_printer == 0):
            print("NumCorrect:", numCorrect, "with total:", total, " (Batch", batch, ")")
      return numCorrect / total

In [94]:
print(len(gen_train))
training_acc = testModel(model, gen_train)
print('Training acc:', training_acc)

4
Training acc: 1.0


In [0]:
def predictTest(model, testing_data, sample_only=False):
    image_list = copy.deepcopy(train_dataset.image_list)
    predictions = []
    jpg = []
    lenTest = len(testing_data) * 1.0
    progress_print = 10
    for index, (image, label, imageName) in enumerate(testing_data):
          image = image.to(device)
          predicted = model(image)
          top5Pred, top5PredIndices = torch.topk(predicted, 5)
          for p in range(len(top5PredIndices)):
              currPred = top5PredIndices[p]
              currPred.cpu()
              relabel_whaleid = []
              for i in currPred:          
                  relabel_whaleid.append(weight_df.loc[int(i),'Id'])
              jpg.append(imageName[p])
              predictions.append(' '.join(relabel_whaleid))  
          if index % progress_print == 0:
            print(f'{index / lenTest * 100}%')
          if sample_only:
            break;
    
    whale_predictions = pd.DataFrame({'Image':jpg,'Id':predictions})
    return whale_predictions

In [102]:
test_predictions = predictTest(model, gen_train, False)
test_predictions = test_predictions[test_predictions.columns[::-1]]
test_predictions.head()

0.0%


,Image,Id
0,ff1355a33.jpg,w_5ae026a new_whale w_ba4f96c w_2548d54 w_789c969
1,ff03a0385.jpg,new_whale w_772845b w_555dcb4 w_2807565 w_7860ed5
2,ff66e1a65.jpg,w_555dcb4 new_whale w_4d06559 w_bfae26b w_8b8dca8
3,ffad62503.jpg,new_whale w_343f088 w_8103832 w_772845b w_bdded04
4,ff75f104a.jpg,new_whale w_d0a7600 w_bdded04 w_107b98e w_2807565
